In [41]:
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [42]:
with open("aging_data_comparisson.json") as json_data:
    data = json.load(json_data)

normalized_data = []

for datum in data:
    new_datum = {key: datum[key] for key in datum if key != "results"}
    prefixes = ["early_adult", "middle_aged_adult", "late_adult"]
    columns = ["significance", "pipeline_stable_id", "procedure_stable_id", "parameter_stable_id", "mp_terms"]
    for result in datum["results"]:
        current_prefix = result["life_stage"].lower().replace(" ", "_")
        for prefix in prefixes:
            for column in columns:
                if  f"{prefix}_{column}" not in new_datum:
                    if current_prefix == prefix and column in result:
                        new_datum[prefix + "_" + column] = result[column]
    normalized_data.append(new_datum)
# for d in normalized_data:
#     if d["early_adult_significance"] != None and d["late_adult_significance"] != None:
#         print(d)
normalized_data = [d for d in normalized_data if "early_adult_significance" in d and "late_adult_significance" in d and "marker_symbol" in d]


In [43]:
from matplotlib.colors import LinearSegmentedColormap


df = pd.DataFrame(normalized_data)
conditions = [
 (df["early_adult_significance"] == False) & (df["late_adult_significance"] == False),
 (df["early_adult_significance"] == True) & (df["late_adult_significance"] == True),
 (df["early_adult_significance"] == False) & (df["late_adult_significance"] == True),
 (df["early_adult_significance"] == True) & (df["late_adult_significance"] == False)
]
choices = [1, 2, 3, 4]

df["score"] = np.select(conditions, choices, default=0)
df["procedure_parameter"] = df["procedure_name"] + " - " + df["parameter_name"]
heat_map_data = df.pivot_table( "score", "marker_symbol", "procedure_name", aggfunc='max')
data = heat_map_data.transpose().to_dict()
nivo_heatmap_data = []
for gene_symbol, datum in data.items():
    nivo_heatmap_data.append({"id": gene_symbol, "data": [{"x": k, "y": v if not pd.isna(v) else 0} for k,v in datum.items()]})

with open('gene_vs_procedure.json', 'w') as f:
    json.dump(nivo_heatmap_data, f)




In [44]:
sns.set(font_scale=0.8)
# For only three colors, it's easier to choose them yourself.
# If you still really want to generate a colormap with cubehelix_palette instead,
# add a cbar_kws={"boundaries": linspace(-1, 1, 4)} to the heatmap invocation
# to have it generate a discrete colorbar instead of a continous one.
myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))
plt.subplots(figsize=(100,150))
ax = sns.heatmap(heat_map_data, cmap=cmap, linewidths=.5, linecolor='lightgray', annot=True)

# Manually specify colorbar labelling after it's been generated
colorbar = ax.collections[0].colorbar
colorbar.set_ticks([-0.667, 0, 0.667])
colorbar.set_ticklabels(['B', 'A', 'C'])

# X - Y axis labels
ax.set_ylabel('FROM')
ax.set_xlabel('TO')

# Only y-axis labels need their rotation set, x-axis labels already have a rotation of 0
_, labels = plt.yticks()
plt.setp(labels, rotation=0)

plt.show()
